In [1]:
Train_data = '/Users/ranjanumeshrao/Downloads/income_prediction/agro_zone_cols.csv'

In [2]:
import pandas as pd
df = pd.read_csv(Train_data)

In [8]:
# Get columns with data types other than int and float
nnc = df.select_dtypes(exclude=['int', 'float'])

# Print column names and their corresponding data types
for col, dtype in nnc.dtypes.items():
    print(f"Column: {col}, Data Type: {dtype}")


#['Kharif Seasons  Agro Ecological Sub Zone in 2022', 'Rabi Seasons Agro Ecological Sub Zone in 2022', 'Rabi Seasons Agro Ecological Sub Zone in 2021','Kharif Seasons Agro Ecological Sub Zone in 2021', 'Kharif Seasons Agro Ecological Sub Zone in 2020', 'Rabi Seasons Agro Ecological Sub Zone in 2020']

Column: K022-Nearest Mandi Name, Data Type: object
Column: K022-Ambient temperature (min & max), Data Type: object
Column: R022-Ambient temperature (min & max), Data Type: object
Column: K021-Ambient temperature (min & max), Data Type: object
Column: R021-Ambient temperature (min & max), Data Type: object
Column: R020-Ambient temperature (min & max), Data Type: object


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Convert State names to integer labels
label_encoder = LabelEncoder()
df['State_ID'] = label_encoder.fit_transform(df['State'])

# Define Embedding Layer (Assume 3 embedding dimensions)
num_states = len(df['State_ID'].unique())  # Total unique states
embedding_dim = 1

# Create Embedding Model
state_input = tf.keras.Input(shape=(1,), dtype=tf.int32, name="State_Input")
state_embedding = tf.keras.layers.Embedding(input_dim=num_states, output_dim=embedding_dim)(state_input)
state_embedding = tf.keras.layers.Flatten()(state_embedding)

model = tf.keras.Model(inputs=state_input, outputs=state_embedding)

# Generate Embeddings for Each Unique State_ID
unique_state_ids = np.array(df['State_ID'].unique()).reshape(-1, 1)
embedding_values = model.predict(unique_state_ids)

# Create a Mapping Dictionary (State_ID → Embedding Vector)
embedding_dict = {state_id: embedding for state_id, embedding in zip(df['State_ID'].unique(), embedding_values)}


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


In [5]:
def replace_with_embedding(state_id):
    return embedding_dict[state_id]

df['State_emb'] = df['State_ID'].apply(lambda x: replace_with_embedding(x)[0])  # Extract scalar value

# Drop the original columns
df.drop(columns=['State_ID'], inplace=True)

# Display updated DataFrame
df["State_emb"]


0       -0.032610
1       -0.032610
2        0.022126
3       -0.032610
4        0.023553
           ...   
53301   -0.025933
53302   -0.025933
53303   -0.025933
53304   -0.025933
53305   -0.025933
Name: State_emb, Length: 53306, dtype: float32

In [8]:
from sklearn.preprocessing import LabelEncoder

# Define columns to apply label encoding
categorical_columns = ['REGION', 'SEX', 'CITY', 'DISTRICT', 'VILLAGE', 'MARITAL_STATUS', 
                       'Location', 'Address type', 'Ownership']

# Apply label encoding
label_encoder = LabelEncoder()

for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

print(df[categorical_columns].head())  # View the transformed columns


   REGION  SEX  CITY  DISTRICT  VILLAGE  MARITAL_STATUS  Location  \
0       0    1   250       308     4924               0     34002   
1       0    1  1727        19        8               0     27662   
2       1    1   214       258     3940               0     34002   
3       0    1  1566       241     1066               0     34002   
4       4    1  2228       221     4667               0     34002   

   Address type  Ownership  
0             3          3  
1             2          0  
2             3          3  
3             3          3  
4             3          3  


In [ ]:
# ['Village category based on socio-economic parameters (Good, Average, Poor)','R022-Village category based on Agri parameters (Good, Average, Poor)','K022-Village category based on socio-economic parameters (Good, Average, Poor)','K022-Village category based on Agri parameters (Good, Average, Poor)']

from sklearn.preprocessing import LabelEncoder

# List of columns to be label encoded
columns_to_encode = [' Village category based on socio-economic parameters (Good, Average, Poor)', 
                     'R022-Village category based on Agri parameters (Good, Average, Poor)', 
                     'K022-Village category based on socio-economic parameters (Good, Average, Poor)', 
                     'K022-Village category based on Agri parameters (Good, Average, Poor)']

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply Label Encoding and shift the encoded values by 1
for col in columns_to_encode:
    df[col] = label_encoder.fit_transform(df[col]) + 1  # Adding 1 to start from 1


In [21]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input
from tensorflow.keras.models import Model
import pandas as pd

# Example dataframe (df) and soil columns
soil_columns = ['Kharif Seasons  Type of soil in 2022', 
                'Rabi Seasons Type of soil in 2022', 
                'Rabi Seasons Type of soil in 2021', 
                'Kharif Seasons Type of soil in 2021', 
                'Kharif Seasons Type of soil in 2020', 
                'Rabi Seasons Type of soil in 2020']

# Function to apply embedding encoding
def apply_embedding_encoding(df, columns, embedding_dim=1):
    for col in columns:
        # Apply StringLookup to convert soil type to integers
        label_encoder = tf.keras.layers.StringLookup(output_mode='int', mask_token=None)
        label_encoder.adapt(df[col])  # Fit the encoder to the column
        
        # Encoding soil types to integer labels
        df[col + '_encoded'] = label_encoder(df[col])
        
        # Create an embedding layer for this column
        input_layer = Input(shape=(1,))
        embedding_layer = Embedding(input_dim=len(label_encoder.get_vocabulary()), 
                                    output_dim=embedding_dim, 
                                    input_length=1, 
                                    trainable=True)(input_layer)
        
        # Build a simple model to train the embedding
        model = Model(inputs=input_layer, outputs=embedding_layer)
        
        # Train the model briefly on this data
        model.compile(optimizer='adam', loss='mse')
        model.fit(df[col + '_encoded'].values, df[col + '_encoded'].values, epochs=1, batch_size=64, verbose=0)
        
        # Extract the embeddings from the trained model
        embeddings = model.predict(df[col + '_encoded'].values)
        
        # Convert the embeddings into a DataFrame
        embedding_df = pd.DataFrame(embeddings.squeeze(), columns=[f"{col}_emb_{i+1}" for i in range(embedding_dim)])
        
        # Concatenate the embeddings to the original dataframe
        df = pd.concat([df, embedding_df], axis=1)
        
        # Optionally, drop the original columns
        df.drop(columns=[col, col + '_encoded'], inplace=True)
    
    return df

# Example usage with embedding_dim=1
df_encoded = apply_embedding_encoding(df, soil_columns, embedding_dim=1)

# Check the result
print(df_encoded.head())

1666/1666 ━━━━━━━━━━━━━━━━━━━━ 0s 242us/step


/opt/anaconda3/envs/Cattle/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1666/1666 ━━━━━━━━━━━━━━━━━━━━ 0s 242us/step


/opt/anaconda3/envs/Cattle/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1666/1666 ━━━━━━━━━━━━━━━━━━━━ 0s 230us/step


...

1666/1666 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step


/opt/anaconda3/envs/Cattle/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1666/1666 ━━━━━━━━━━━━━━━━━━━━ 0s 230us/step


/opt/anaconda3/envs/Cattle/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1666/1666 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step
   Unnamed: 0.1  Unnamed: 0          FarmerID  REGION  SEX  CITY  Zipcode  \
0             0           0  1002818465057450       0    1   250   464668   
1             1           1  1003667148159900       0    1  1727   473443   
2             2           2  1012300674433870       1    1   214   848125   
3             3           3  1013472263587380       0    1  1566   458556   
4             4           4  1019525480704050       4    1  2228   413527   

   DISTRICT  VILLAGE  Address type  ...  \
0       308     4924             3  ...   
1        19        8             2  ...   
2       258     3940             3  ...   
3       241     1066             3  ...   
4       221     4667             3  ...   

    Land Holding Index source (Total Agri Area/ no of people)  \
0                                           0.773129            
1                                           0.199800            
2                                     

In [4]:
# ['Rabi Seasons Type of water bodies in hectares 2022', 'Rabi Seasons Type of water bodies in hectares 2021', 'Kharif Seasons Type of water bodies in hectares 2021', 'Kharif Seasons Type of water bodies in hectares 2020', 'Rabi Seasons Type of water bodies in hectares 2020']

import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding
import tensorflow as tf

# Columns to process
target_columns = [
    'Kharif Seasons Type of water bodies in hectares 2020', 
    'Rabi Seasons Type of water bodies in hectares 2020'
]

# Dictionary to store label mappings for each column
label_mappings = {}

for col in target_columns:
    # Step 1: Extract unique labels dynamically
    unique_labels = set()
    for val in df[col]:
        unique_labels.update(val)

    # Create a label mapping dynamically
    label_mapping = {label: idx for idx, label in enumerate(sorted(unique_labels))}
    label_mappings[col] = label_mapping  # Store mapping for later use
    print(f"\nGenerated Label Mapping for {col}:", label_mapping)

    # Step 2: Encode labels using the dynamically generated mapping
    df[f'{col}_encoded'] = df[col].apply(lambda value: np.array([label_mapping[label] for label in value], dtype=np.int32))

# Step 3: Apply embedding encoding
def apply_embedding_encoding(df, col, embedding_dim=1):
    max_value = max([max(values) for values in df[f'{col}_encoded']]) + 1
    embedding_layer = Embedding(input_dim=max_value, output_dim=embedding_dim, input_length=1, trainable=True)

    embedded_values = []
    for row in df[f'{col}_encoded']:
        row_embedding = np.mean(embedding_layer(np.array(row, dtype=np.int32)).numpy(), axis=0)  # Ensure int32 input
        embedded_values.append(row_embedding.astype(np.float32))  # Ensure float32 output

    # Convert list of arrays into a proper numerical format
    df[f'{col}_emb'] = np.vstack(embedded_values)

    return df

# Apply embedding encoding for each target column
for col in target_columns:
    df = apply_embedding_encoding(df, col, embedding_dim=1)
    df.drop(columns=[f'{col}_encoded', col], inplace=True)  # Drop unnecessary columns

# Check the final DataFrame
print("\nFinal Data with Embeddings:\n", df.head())
print("\nData Types:\n", df.dtypes)


Generated Label Mapping for Kharif Seasons Type of water bodies in hectares 2020: {' ': 0, "'": 1, ',': 2, 'N': 3, '[': 4, ']': 5, 'a': 6, 'b': 7, 'd': 8, 'e': 9, 'i': 10, 'k': 11, 'l': 12, 'n': 13, 'o': 14, 'r': 15, 's': 16, 't': 17, 'v': 18, 'w': 19}

Generated Label Mapping for Rabi Seasons Type of water bodies in hectares 2020: {' ': 0, "'": 1, ',': 2, 'N': 3, '[': 4, ']': 5, 'a': 6, 'b': 7, 'd': 8, 'e': 9, 'i': 10, 'k': 11, 'l': 12, 'n': 13, 'o': 14, 'r': 15, 's': 16, 't': 17, 'v': 18, 'w': 19}


/opt/anaconda3/envs/Cattle/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/opt/anaconda3/envs/Cattle/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Final Data with Embeddings:
    Unnamed: 0.5  Unnamed: 0.4  Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  \
0             0             0             0             0             0   
1             1             1             1             1             1   
2             2             2             2             2             2   
3             3             3             3             3             3   
4             4             4             4             4             4   

   Unnamed: 0          FarmerID  REGION  SEX  CITY  ...  State_emb  \
0           0  1002818465057450       0    1   250  ...  -0.032610   
1           1  1003667148159900       0    1  1727  ...  -0.032610   
2           2  1012300674433870       1    1   214  ...   0.022126   
3           3  1013472263587380       0    1  1566  ...  -0.032610   
4           4  1019525480704050       4    1  2228  ...   0.023553   

   Kharif Seasons  Type of soil in 2022_emb_1  \
0                                    2.127195   


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

agro_zone_cols = [
    'Kharif Seasons  Agro Ecological Sub Zone in 2022',
    'Rabi Seasons Agro Ecological Sub Zone in 2022',
    'Rabi Seasons Agro Ecological Sub Zone in 2021',
    'Kharif Seasons Agro Ecological Sub Zone in 2021',
    'Kharif Seasons Agro Ecological Sub Zone in 2020', 
    'Rabi Seasons Agro Ecological Sub Zone in 2020'
]

# Function to apply model-based embedding encoding to all specified columns
def apply_model_based_embedding_encoding(df, columns, embedding_dim=1):
    for col in columns:
        # Step 1: Convert the categorical values to numeric labels
        unique_labels = df[col].unique()
        label_mapping = {label: idx for idx, label in enumerate(sorted(unique_labels))}
        df[col + '_encoded'] = df[col].map(label_mapping).astype(np.int32)

        # Step 2: Define and train a simple neural network model for embeddings
        inputs = layers.Input(shape=(1,))
        embedding_layer = layers.Embedding(input_dim=len(unique_labels), output_dim=embedding_dim, input_length=1, trainable=True)(inputs)
        model = models.Model(inputs, embedding_layer)

        # Step 3: Compile and train the model
        model.compile(optimizer='adam', loss='mse')
        # For training, the model uses its own encoded column as target (for simplicity)
        # You can use another column or some other target as per your use case
        model.fit(df[col + '_encoded'], df[col + '_encoded'], epochs=5, verbose=0)

        # Step 4: Generate embeddings for each label using the trained model
        embeddings = model.predict(df[col + '_encoded'], batch_size=64)
        df[col + '_embedding'] = embeddings.flatten()

    return df

# Apply model-based embedding encoding
df_encoded = apply_model_based_embedding_encoding(df, agro_zone_cols, embedding_dim=1)
df_encoded.drop(columns=[col, col + '_encoded'], inplace=True)


print(df_encoded.head())
print("\nData Types:\n", df_encoded.dtypes)

/opt/anaconda3/envs/Cattle/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 247us/step


/opt/anaconda3/envs/Cattle/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 245us/step


/opt/anaconda3/envs/Cattle/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


...

833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step


/opt/anaconda3/envs/Cattle/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step


/opt/anaconda3/envs/Cattle/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step
   Unnamed: 0.6  Unnamed: 0.5  Unnamed: 0.4  Unnamed: 0.3  Unnamed: 0.2  \
0             0             0             0             0             0   
1             1             1             1             1             1   
2             2             2             2             2             2   
3             3             3             3             3             3   
4             4             4             4             4             4   

   Unnamed: 0.1  Unnamed: 0          FarmerID  REGION  SEX  ...  \
0             0           0  1002818465057450       0    1  ...   
1             1           1  1003667148159900       0    1  ...   
2             2           2  1012300674433870       1    1  ...   
3             3           3  1013472263587380       0    1  ...   
4             4           4  1019525480704050       4    1  ...   

   Kharif Seasons  Agro Ecological Sub Zone in 2022_embedding  \
0                                     

In [10]:
df.to_csv("/Users/ranjanumeshrao/Downloads/income_prediction/agro_zone_cols_final.csv")